# 初始化

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from dotenv import load_dotenv
load_dotenv(dotenv_path='', override=True)

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model='deepseek-chat', api_key=os.getenv('DEEPSEEK_API_KEY'))

# 时间回溯

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
tool = TavilySearch(max_results=2)

llm_with_tools = llm.bind_tools([tool])

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

In [ ]:
graph_builder.add_edge(START, "chatbot")

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")

In [ ]:
memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print("displsy error")

### 添加步骤

向你的图中添加步骤。每一步都将在其状态历史中被创建检查点。

In [ ]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

### 3. 回放完整的状态历史¶
现在你已经为聊天机器人添加了步骤，你可以 `replay` (回放) 完整的状态历史，以查看所有发生过的事情。

In [ ]:
to_replay = None

for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        print("="*30)
        to_replay = state
        print(to_replay)
        print("="*30)

In [ ]:
print(to_replay.next)
print(to_replay.config)

### 4. 从特定时间点加载状态¶
检查点的 to_replay.config 包含一个 `checkpoint_id` 时间戳。

提供这个 `checkpoint_id` 值会告诉 LangGraph 的检查点管理器从那个时间点**加载**状态。

In [ ]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()